<a href="https://colab.research.google.com/github/RomainHugues/Ship_simulator/blob/master/ship_simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installs and imports

In [1]:
import pandas

try:
    import geopandas
except ImportError:
    !pip install geopandas
    import geopandas


import xarray

try:
    import cfgrib
except ImportError:
    !sudo apt-get install libeccodes0
    !pip install cfgrib
    import cfgrib

import numpy
from scipy import interpolate
import os

from datetime import datetime, timedelta
import shapely
import random
import time

try:
    import geemap.eefolium as geemap
except ImportError:
    !pip install geemap
    import geemap.eefolium as geemap

import ee

from matplotlib import pyplot as plt

from google.colab import drive

     |████████████████████████████████| 972kB 3.0MB/s 
     |████████████████████████████████| 14.8MB 230kB/s 
     |████████████████████████████████| 10.9MB 38.3MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libeccodes-data
The following NEW packages will be installed:
  libeccodes-data libeccodes0
0 upgraded, 2 newly installed, 0 to remove and 11 not upgraded.
Need to get 2,861 kB of archives.
After this operation, 66.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libeccodes-data all 2.6.0-2 [1,260 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libeccodes0 amd64 2.6.0-2 [1,601 kB]
Fetched 2,861 kB in 1s (2,709 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialo


Environment classes : Wind and Current

In [ ]:
class Wind:

    def __init__(self):
        
        self.dimensions = None
        self.latitude = None
        self.longitude = None
        self.timestep = None
        self.wind_data = None

    def acquire_grib_data(self, grib_file_location):

        wind_dataset = cfgrib.open_dataset(grib_file_location)

        # Grid dimension
        self.dimensions = numpy.array([wind_dataset.dims['step'], wind_dataset.dims['latitude'], wind_dataset.dims['longitude']])
        # Grid coordinates
        self.latitude = wind_dataset.coords['latitude']
        self.longitude = wind_dataset.coords['longitude']
        self.timestep = wind_dataset.coords['step']
        
        # Grid data
        self.wind_data = wind_dataset.data_vars

    def get_wind_info_from_position(self, position, date):

        # Eastwards component
        wind_position_u = wind_forecast.wind_data['u10'].interp(latitude = position.coordinates[0], 
                                                            longitude = position.coordinates[1],
                                                            step = (date - wind_forecast.wind_data['u10'].time))

        # Northwards component
        wind_position_v = wind_forecast.wind_data['v10'].interp(latitude = position.coordinates[0], 
                                                            longitude = position.coordinates[1],
                                                            step = (date - wind_forecast.wind_data['v10'].time))

        wind_direction = (90 - numpy.arctan2(wind_position_v, wind_position_u) * 180 / numpy.pi) % 360
        wind_speed = numpy.sqrt(wind_position_u**2 + wind_position_v**2)

        return wind_speed.data, wind_direction.data 

class Current:

    def __init__(self):

        current_field = None

    def acquire_current_data(self):

        return 0

    def delta_speed_from_position(self):

        return 0
    

Position class

In [ ]:
class Position:

    def __init__(self, lat, lon):

        self.coordinates = numpy.array([lat,lon],dtype=numpy.float32) 

    def update(self, heading, speed, delta_T=60):
        
        R = 6371e3
        # Update position for ship bearing heading with given speed during delta_T seconds
        [phi1, lambda1] = self.coordinates * numpy.pi / 180
        # distance in meters
        distance = (speed * 1852 / 3600) * delta_T
        # Angular distance
        delta = distance / R

        #Bearing
        theta = heading * numpy.pi / 180

        phi2 = numpy.arcsin(numpy.sin(phi1)*numpy.cos(delta)+numpy.cos(phi1)*numpy.sin(delta)*numpy.cos(theta)) * 180 / numpy.pi
        lambda2 = (lambda1 + numpy.arctan2(numpy.sin(theta)*numpy.sin(delta)*numpy.cos(phi1), numpy.cos(delta)-numpy.sin(phi1)*numpy.sin(phi2))) * 180 / numpy.pi
        
        self.coordinates = numpy.array([phi2,lambda2])


Ship and NAvigator class

In [ ]:
class Ship:

    def __init__(self, starting_point, starting_date):

        self.heading = 0
        self.position = starting_point
        self.date = starting_date
        self.speed = 0
        self.polar_best_speed = None
        self.wind_relative_angle = 0
        self.heading_track = []
        self.position_track = []

    def update_date(self, delta_T_seconds):

        self.date += timedelta(seconds=delta_T_seconds) 

    def acquire_polar_data(self,polar_data_file):

        polar_csv = pandas.read_csv(polar_file_location)

        polar_speed = numpy.zeros([181,71,5])

        for i in range(181*71*5-1):
            polar_speed[int(polar_csv.iloc[i, 2]),int(polar_csv.iloc[i, 0]),int(polar_csv.iloc[i, 1]) - 1] = polar_csv.iloc[i, 3]

        polar_best_sail = numpy.argmax(polar_speed, axis=2).squeeze()
        polar_best_speed = numpy.max(polar_speed, axis=2).squeeze()

        # Interpolation function
        self.polar_best_speed = interpolate.interp2d(y=numpy.arange(181),
                                                           x=numpy.arange(71),
                                                           z=polar_best_speed,
                                                           kind='cubic')

    def get_speed(self, wind_speed, wind_direction):

        # Wind relative angle
        wind_relative_angle = numpy.abs((self.heading - wind_direction) % 180)

        self.wind_relative_angle = wind_relative_angle

        # Ship speed
        self.speed = round(self.polar_best_speed(wind_speed, wind_relative_angle)[0],2)

    def set_heading(self,new_heading):

        self.heading = new_heading
        self.heading_track.append(self.heading)

    def set_starting_point(self,starting_point):

        self.position = starting_point
        self.position_track.append(self.position.coordinates)

    def update_position(self, delta_T):

        self.position.update(self.heading, self.speed, delta_T)
        self.position_track.append(self.position.coordinates)

class Navigator:

    def __init__(self):

        self.heading = 0

    def heading_update(heading):

        # Heading is bearing to next checkpoint
        self.heading = 0

    def best_VMG_heading_update():

        # Heading is bearing maximizing VMG
        self.heading = 0


Road class

In [ ]:
class Road:

    def __init__(self):

        self.checkpoint_list = []
        self.bearing_list = None
        self.distances_list = None
        self.minimal_distance = None
        self.route_status = 0

        self.bearing_to_next_checkpoint = 0
        self.distance_to_next_checkpoint = 0

    def set_starting_point(self,point):

        self.checkpoint_list = [point]

    def add_checkpoint(self,point):

        self.checkpoint_list.append(point)

    def remove_checkpoint(self,index):

        self.checkpoint_list.pop(index)

    def insert_checkpoint(self,index, point):

        self.checkpoint_list.insert(index, point)

    def bearing_and_distance(self):

        # compute bearing and distance from starting point to endpoint through all checkpoints

        R = 6371e3
        array_road = numpy.stack(self.checkpoint_list)

        point_1 = array_road[:-1,:]
        point_2 = array_road[1:,:]

        phi1 = point_1[:,0] * numpy.pi / 180
        phi2 = point_2[:,0] * numpy.pi / 180
        delta_phi = phi2 - phi1
        lambda1 = point_1[:,1] * numpy.pi / 180
        lambda2 = point_2[:,1] * numpy.pi / 180
        delta_lambda = lambda2 - lambda1

        # Distance

        a = numpy.sin(delta_phi/2)**2+numpy.cos(phi1)*numpy.cos(phi2)*numpy.sin(delta_lambda/2)**2
        c = 2 * numpy.arctan2(numpy.sqrt(a), numpy.sqrt(1-a))
        d = R * c

        # Distance list in nautical miles
        self.distance_list = d / 1852

        self.minimal_distance = numpy.sum(self.distance_list)

        # Bearing

        y = numpy.sin(lambda2-lambda1) * numpy.cos(phi2)
        x = numpy.cos(phi1)*numpy.sin(phi2) - numpy.sin(phi1)*numpy.cos(phi2)*numpy.cos(lambda2-lambda1)
        theta = numpy.arctan2(y, x)

        # Bearing list in degrees
        self.bearing_list = (theta * 180 / numpy.pi + 360) % 360

    def bearing_and_distance_to_checkpoint(self, position):
        
        R = 6371e3

        # Compute distance to next checkpoint
        current_position = position.coordinates
        next_checkpoint = self.checkpoint_list[self.route_status+1]

        phi1 = current_position[0] * numpy.pi / 180
        phi2 = next_checkpoint[0] * numpy.pi / 180
        delta_phi = phi2 - phi1
        lambda1 = current_position[1] * numpy.pi / 180
        lambda2 = next_checkpoint[1] * numpy.pi / 180
        delta_lambda = lambda2 - lambda1

        # Distance
        a = numpy.sin(delta_phi/2)**2+numpy.cos(phi1)*numpy.cos(phi2)*numpy.sin(delta_lambda/2)**2
        c = 2 * numpy.arctan2(a**0.5, (1-a)**0.5)
        d = R * c

        # Distance list in nautical miles
        self.distance_to_next_checkpoint =round(d / 1852,3)
        
        # Bearing
        y = numpy.sin(lambda2-lambda1) * numpy.cos(phi2)
        x = numpy.cos(phi1)*numpy.sin(phi2) - numpy.sin(phi1)*numpy.cos(phi2)*numpy.cos(lambda2-lambda1)
        theta = numpy.arctan2(y, x)

        # Bearing list in degrees
        self.bearing_to_next_checkpoint = (theta * 180 / numpy.pi + 360) % 360

    def checkpoint_crossing(self, speed, delta_T):

        if self.distance_to_next_checkpoint <= (speed * delta_T/3600):
            # Checkpoint is reached, move to next checkpoint
            self.route_status += 1
            

Main Script text-only scenario

In [ ]:
# Initialize Wind

# https://marine.meteoconsult.fr/cartes-meteo-marine/fichiers-grib.php 09 Manche Area 2020-09-19 8:20
grib_file_location = '/content/drive/My Drive/sailing_simulator/uLsEcMxxOoSYmtRzKDl0e75I4HAjqqDApv_.grb'

wind_forecast = Wind()
wind_forecast.acquire_grib_data(grib_file_location)


# Initialize Road : From Southampton to St Malo through Weymouth. Departure at 10:30 Sunday 19th September 2020
starting_date = pandas.to_datetime('2020-09-19 10:00:00')
starting_point = Position(50.931429, -1.367399)
middle_point = Position(50.578713, -2.506107)
end_point = Position(48.6465, -2.0066)

cruise = Road()
cruise.set_starting_point(starting_point.coordinates)
cruise.add_checkpoint(middle_point.coordinates)
cruise.add_checkpoint(end_point.coordinates)
cruise.bearing_and_distance()
delta_T = 300

n_checkpoints = len(cruise.checkpoint_list)

# Initialize Ship

# IMOCA foil 2019 polar curve
polar_file_location = '/content/drive/My Drive/sailing_simulator/polaires_imoca60_foil_2019.csv'

valou_crusader = Ship(starting_point, starting_date)
valou_crusader.acquire_polar_data(polar_file_location)

# Navigator
navigator = Navigator()

condition = True
t = 0

while condition:

    # Distance and heading to next checkpoint
    cruise.bearing_and_distance_to_checkpoint(valou_crusader.position)

    # Display
    if t*delta_T % 3600 == 0:
        print(t, valou_crusader.date, cruise.bearing_to_next_checkpoint, 
            cruise.distance_to_next_checkpoint, 
            valou_crusader.speed, 
            valou_crusader.position.coordinates,
            cruise.route_status)

    # Set heading following shortest track policy
    valou_crusader.heading = cruise.bearing_to_next_checkpoint

    # Wind speed and direction from position
    wind_speed, wind_direction = wind_forecast.get_wind_info_from_position(valou_crusader.position, valou_crusader.date)

    # Ship speed at position from wind
    valou_crusader.get_speed(wind_speed, wind_direction)

    # Check pointcrossing
    cruise.checkpoint_crossing(valou_crusader.speed, delta_T)

    # Ship displacement
    valou_crusader.update_position(delta_T=delta_T)

    # Update date
    valou_crusader.update_date(delta_T)

    t += 1

    # check if arrived
    if cruise.route_status == n_checkpoints - 1:
        print('You arrived !')
        break


0 2020-09-19 10:00:00 244.35432238805038 48.158 0 [50.93143  -1.367399] 0
12 2020-09-19 11:00:00 244.4784813692824 47.148 1.05 [50.9223921  -1.39013354] 0
24 2020-09-19 12:00:00 244.5635403411011 46.492 0.63 [50.91650999 -1.40487072] 0
36 2020-09-19 13:00:00 244.61318211512656 46.122 0.33 [50.91318029 -1.41319561] 0
48 2020-09-19 14:00:00 244.6640069029815 45.753 3.83 [50.90986122 -1.42147012] 0
60 2020-09-19 15:00:00 245.11848081535322 42.793 3.73 [50.88304184 -1.48787472] 0
72 2020-09-19 16:00:00 245.40588856736588 41.158 0.1 [50.86814753 -1.52450943] 0
84 2020-09-19 17:00:00 245.4404886082258 40.97 0.37 [50.86643293 -1.52872738] 0
96 2020-09-19 18:00:00 245.51580706636545 40.567 0.67 [50.86276292 -1.53774953] 0
108 2020-09-19 19:00:00 245.6336272839946 39.956 0.92 [50.85718309 -1.55145397] 0
120 2020-09-19 20:00:00 245.79169993493562 39.168 1.16 [50.84998205 -1.56912353] 0
132 2020-09-19 21:00:00 246.03466498557017 38.021 1.86 [50.83949186 -1.59484021] 0
144 2020-09-19 22:00:00 246.

KeyError: ignored